# 디스크립터

## 객체 속성에 접근할때 발동하는 스페셜 메소드

`__getattribute__` 겟 어트리뷰트

`__getattr__` > 겟어트리

`__setattr__`

`__delattr__`

# `__getattribute__`

속성을 탐색

In [232]:
%%writefile d1.py


class A:
    x = 3
    def __getattribute__(self, name):
        print("getattribute!!!")
        return super().__getattribute__(name)
    
a = A()
a.y = 123

print(a.__dict__)
print(type(a).__dict__)
print(A.__dict__)


print(a.x)
print(a.y)


Overwriting d1.py


In [233]:
!python d1.py

getattribute!!!
{'y': 123}
{'__module__': '__main__', 'x': 3, '__getattribute__': <function A.__getattribute__ at 0x10b85a378>, '__dict__': <attribute '__dict__' of 'A' objects>, '__weakref__': <attribute '__weakref__' of 'A' objects>, '__doc__': None}
{'__module__': '__main__', 'x': 3, '__getattribute__': <function A.__getattribute__ at 0x10b85a378>, '__dict__': <attribute '__dict__' of 'A' objects>, '__weakref__': <attribute '__weakref__' of 'A' objects>, '__doc__': None}
getattribute!!!
3
getattribute!!!
123


In [32]:
%%writefile d2.py

import warnings


class Private:
    def __getattribute__(self, name, perm=False):
        if not perm:
            warnings.warn("속성에 접근할 수 없습니다.")
            return None
        return super().__getattribute__(name)

    
class ScretBox(Private):
    pass


a = ScretBox()
a.x = 3

print(a.x)
print(a.__dict__)
print(type(a).__dict__)


### 숨겨진 값을 찾기 위해서..
### 허가를 거치는 단계

print(type(a).__getattribute__(a, '__dict__', True))


from functools import partial

perm_a = partial(type(a).__getattribute__, perm=True)
print('partial 이용 : ',perm_a(a, 'x'))

Overwriting d2.py


In [33]:
!python d2.py

d2.py:8: UserWarning: 속성에 접근할 수 없습니다.
  warnings.warn("속성에 접근할 수 없습니다.")
None
None
{'__module__': '__main__', '__doc__': None}
{'x': 3}
partial 이용 :  3


# `__getattr__`

객체 속성을 찾을때...
**가장 마지막에 실행**

In [37]:
%%writefile d3.py


import warnings


class A:
    def __getattr__(self, name):
        warnings.warn("값이 없습니다. 디폴트값을 넘깁니다.")
        return 3

a = A()
a.var = 123
print('a.var', a.var)
print('a.xxx', a.xxx)

Overwriting d3.py


In [38]:
!python d3.py

a.var 123
d3.py:8: UserWarning: 값이 없습니다. 디폴트값을 넘깁니다.
  warnings.warn("값이 없습니다. 디폴트값을 넘깁니다.")
a.xxx 3


# `__getattribute__` vs `__getattr__`

In [41]:
%%writefile d4.py


class A:
    def __getattribute__(self, name):
        print("getattribute")
        return super().__getattribute__(name)
    
    def __getattr__(self, name):
        print("getattr")
        return '없습니다.'
    

a = A()

print(a.no)

a.var = 1

print(a.var)

Overwriting d4.py


In [42]:
!python d4.py

getattribute
getattr
없습니다.
getattribute
1


# class 속성을 탐색할때

In [45]:
%%writefile d5.py


class Meta(type):
    def __getattribute__(self, name):
        return "classattribute"
    
    def __getattr__(self, name):
        return "classattr"
    

class A(metaclass=Meta):
    pass


#print(A.b)
a = A()
print(a.b)

Overwriting d5.py


In [46]:
!python d5.py

Traceback (most recent call last):
  File "d5.py", line 17, in <module>
    print(a.b)
AttributeError: 'A' object has no attribute 'b'


# `__setattr__`

속성에 객체를 할당할때

In [48]:
%%writefile d6.py


class A:
    def __setattr__(self, name, value):
        # return 필요 없음
        print("setattr!!")
        self.__dict__[name] = value

a = A()
a.var = 3
print(a.var)

Overwriting d6.py


In [49]:
!python d6.py

setattr!!
3


# `__setattr__` 적용

In [50]:
%%writefile d7.py

import sys
from types import ModuleType


class MyModule(ModuleType):
    def __repr__(self):
        return 'MyModule'
    
    def __setattr__(self, name, value):
        print(name, "세팅 불가")
        
    @staticmethod
    def func(a, b):
        pass
        
def func(a, b):
    pass
        
sys.modules[__name__].__class__ = MyModule

Writing d7.py


In [53]:
%%writefile d8.py

import d7

print(d7)
d7.a = 3
print(d7.a)

Overwriting d8.py


In [54]:
!python d8.py

MyModule
a 세팅 불가
Traceback (most recent call last):
  File "d8.py", line 6, in <module>
    print(d7.a)
AttributeError: module 'd7' has no attribute 'a'


# `__delattr__`

In [57]:
%%writefile d9.py


class A:
    def __delattr__(self, name):
        print("지울 수 없습니다.")
        
a = A()
a.var = 3

print(a.var)

del a.var # 여기서 __delattr__가 작동

print(a.var)

Overwriting d9.py


In [58]:
!python d9.py

3
지울 수 없습니다.
3


## `__delattr__` 활용 예

In [59]:
...

Ellipsis

# 스페셜메소드를 통한 속성 컨트롤

### 인스턴스를 생성할때 반지름을 입력받는 원 객체 / 원의 속성을 가지고 있는..

> 반지름은 mutable    
> 그 외에 지름, 둘레, 넓이는 immutable

In [65]:
%%writefile d10.py


import warnings

MUTABLE = ['radius']
IMMUTABLE = ['diameter', 'circumference', 'area']

class Circle:
    pi = 3.1415
    
    def __init__(self, r):
        self.radius = r
        
    def diameter(self):
        return self.radius * 2
    
    def circumference(self):
        return self.radius * self.pi * 2
    
    def area(self):
        return (self.radius ** 2) * self.pi
    
    def __setattr__(self, name, value):
        if name in IMMUTABLE:
            warnings.warn("불가")
        return super().__setattr__(name, value)
    
    # __delattr__ 추가
    

c = Circle(10)
print(c.radius)
print(c.diameter())
c.radius = 100
c.area = 123

Overwriting d10.py


In [66]:
!python d10.py

10
20
d10.py:25: UserWarning: 불가
  warnings.warn("불가")


# 문제점 : 속성 개별 특성을 부여하기 힘들다는 단점

# 해결책 : 디스크립터

## 디스크립터란

프로토콜, 규약

`__get__`, `__set__`, `__delete__` 중 하나가 구현되어 있는 클래스를 디스크립터 클래스

In [67]:
class Descriptor:
    def __init__(self, value):
        self.value = value
        
    def __get__(self, instance, owner):
        print("get")
        return self.value
    
    def __set__(self, instance, value):
        print("set")
        self.value = value
        
    def __delete__(self, instance):
        print("delete")
        del self.value
        
class A:
    value = Descriptor(10) # instance

## `__get__`

In [68]:
a = A()

In [71]:
# type(a).__dict__['value'].__get__(a, type(a)) 와 같은
a.value # __get__ 

get


10

## `__set__`

In [74]:
a.value = 12  # == type(a).__dict__['value'].__set__(a, 12)

set


In [75]:
a.value

get


12

### `__delete__`

In [77]:
del a.value  # type(a).__dict__['value'].__delete__(a)

delete


In [79]:
a.value =123
type(a).__dict__['value'].__delete__(a)

set
delete


## 디스크립터 프로토콜이 없다면..

In [84]:
class D:
    def __getattribute__(self, name):
        print("getattribute")
        return super().__getattribute__(name)

class A:
    value = D()

In [85]:
a = A()
a.value.data = 1

In [86]:
a.value.data

getattribute


1

## `__get__` in 디스크립터

In [87]:
class FirstDescriptor:
    def __init__(self, value):
        self.value = value
        
    def __get__(self, instance, owner):
        print("get")
        return self.value
    

class A:
    a = FirstDescriptor(3)

In [88]:
a = A()
a.a

get


3

## `__set__` 

읽기 전용인것처럼

In [91]:
class Descriptor:
    def __init__(self, value):
        self.value = value
        
    def __get__(self, instance, onwer):
        print("get")
        return self.value
    
#     def __set__(self, instance, value):
#         print("set")
#         self.value = value  # 쓰기 가능
        
    def __set__(self, instance, value):
        # 읽기전용
        print("set")
        raise AttributeError("read only")
        
class A:
    a = Descriptor(3)
    
a = A()
a.a = 12
a.a

set


AttributeError: read only

# 디스크립터에서 객체를 저장하는 위치

## 디스크립터에 저장

In [124]:
class Descriptor:
    def __get__(self, instance, owner): # instance = 클래스 A의 인스턴스
        print("get")
        return self.value # self는 디스크립터 클래스의 인스턴스,
    
    def __set__(self, instance, value):
        print('set')
        self.value = value # self는 디스크립터 클래스의 인스턴스,
        
class A:
    x = Descriptor()

In [125]:
a = A()
a.x = 150

set


In [126]:
a.x

get


150

In [127]:
a.__dict__

{}

In [128]:
type(a).__dict__

mappingproxy({'__dict__': <attribute '__dict__' of 'A' objects>,
              '__doc__': None,
              '__module__': '__main__',
              '__weakref__': <attribute '__weakref__' of 'A' objects>,
              'x': <__main__.Descriptor at 0x10f3534a8>})

In [116]:
b = A()

In [117]:
b.x

get


150

In [100]:
class A:
    x = 3
    
a = A()
b = A()
a.__dict__, b.__dict__

({}, {})

In [101]:
a.x # == a.__dict__['x'] -> type(a).__dict__['x']

3

In [102]:
b.x

3

In [103]:
a.x = 123

In [104]:
a.x

123

In [105]:
type(a).__dict__['x']

3

In [106]:
a.__dict__

{'x': 123}

## 인스턴스에 저장

In [118]:
class descriptor:
    def __get__(self, instane, owner):
        print('get')
        return instance.value
    
    def __set__(self, instance, value):
        print('set')
        instance.value = value
        

class A:
    x = descriptor()

In [119]:
a = A()

In [120]:
a.x = 123

set


In [121]:
b = A()

In [122]:
b.x

get


NameError: name 'instance' is not defined

In [123]:
a.__dict__

{'value': 123}

# 무엇을 저장하느냐..

## 함수를 저장

In [129]:
class descriptor:
    def __init__(self, func):
        self.func = func
        
    def __get__(self, instance, owner):
        print("get")
        return self.func(instance) # 메소드는 첫번째 인자로 self 자기자신을 받아야되기 때문에
    

class A:
    def __init__(self, a, b, c):
        self.a = a
        self.b = b
        self.c = c
    
    @descriptor
    def func(self):
        return self.a + self.b + self.c


In [130]:
a = A(1, 2, 3)

In [132]:
a.func # __call__ X

get


6

## 인자를 받도록..

In [133]:
from functools import partial


class descriptor:
    def __init__(self, func):
        self.func = func
        
    def __get__(self, instance, owner):
        print('get')
        return partial(self.func, instance)
    

class A:
    @descriptor
    def func(self, a, b, c):
        return a + b + c

In [134]:
a = A()

In [135]:
a.func(1, 2, 3)

get


6

# 도전 : 디스크립터로 구현

### 인스턴스를 생성할때 반지름을 입력받는 원 객체 / 원의 속성을 가지고 있는..

> 반지름은 mutable    
> 그 외에 지름, 둘레, 넓이는 immutable

### 함수를 저장하는 방식으로

### sample

```
class MutableAttribute:
    __get__ , __set__, __delete__


class ImmutableAttribute:
    __get__, __set__, __delete__ # immutable 에러발생


class Circle:
    radius = ImMutableAttribute()
    pi = ImMutableAttribute()
    
    @MutableAttribute
    def area(self):
        return self.radius ** 2 * self.pi
        
```

# 디스크립터 메소드 구현시.. 인자는

## `__get__` -> self, instance, owner

## `__set__` -> self, instance, value

## `__delete__` -> self, instance

In [136]:
class MutableAttribute:
    def __init__(self, value=None):
        self.value = value
        
    def __get__(self, instance, owner):
        return self.value
    
    def __set__(self, instance, value):
        self.value = value
        
    def __delete__(self, instance):
        del self.value

        
class ImmutableAttribute:
    def __init__(self, func):
        self.func = func
        
    def __get__(self, instance, owner):
        return self.func(instance)
    
    def __set__(self, instance, value):
        raise AttributeError("read only")
        
    def __delete__(self, instance):
        raise AttributeError("read only")

In [137]:
class Circle:
    pi = 3.1415
    radius = MutableAttribute(10)
    diameter = ImmutableAttribute(lambda self : self.radius * 2)
    
#     @ImmutableAttribute
#     def diameter(self):
#         return self.radius * 2
    
    @ImmutableAttribute
    def circumference(self):
        return self.radius * self.pi * 2
    
    @ImmutableAttribute
    def area(self):
        return self.radius **2 * 2

In [139]:
c = Circle()

In [140]:
c.radius

10

In [141]:
c.diameter

20

In [142]:
c.area

200

In [143]:
c.radius = 100

In [146]:
c.area

20000

In [145]:
c.area = 123

AttributeError: read only

# 도전 : 실행가능 디스크립터


```
class A:
    @descriptor
    def sum(self, a, b, c):
        return a + b + c
        
a = A()
a.sum(1, 2, 3) # 6
```

+ partial 객체를 이용 (functools.partial)

In [150]:
from functools import partial

class descriptor:
    def __init__(self, func):
        self.func = func
        
    def __get__(self, instance, owner):
        # return self.func(instance)
        return partial(self.func, instance)


class A:
    @descriptor
    def sum(self, a, b, c):
        return a + b + c
        
a = A()
a.sum(1, 2, 3) # 6

6

In [151]:
type(a).__dict__['sum'].__get__(a, type(a))(1, 2, 3)

6

In [152]:
class A:
    def sum(self):
        pass

In [155]:
a = A()
type(a).__dict__['sum'].__get__

<method-wrapper '__get__' of function object at 0x10f459c80>

# 디스크립터의 장점

1. 트리거 액션
2. data encapsulation
3. 모든 속성을 class level

# 내장 디스크립터

* property
* classmethod
* staticmethod

## property 사용법

In [161]:
class A:
    def __init__(self, value):
        self.value = value
        
    @property
    def double(self):
        return self.value * 2

In [162]:
a = A(10)

In [163]:
a.value, a.double

(10, 20)

In [164]:
class OnlyInt:

    @property
    def data(self):
        return self._data
    
    @data.setter
    def data(self, value):
        if isinstance(value, int):
            self._data = value
        else:
            raise AttributeError("only int")
            
    @data.deleter
    def data(self):
        raise AttributeError("can't")

In [165]:
t = OnlyInt()

In [166]:
t.data = 12

In [167]:
t.data

12

In [168]:
del t.data

AttributeError: can't

In [169]:
t.data = '123'

AttributeError: only int

property는 표준디스크립터

get, set, delete 로직을 지정할 수 있는..

정보를 instance 저장하게하는..

# 도전 : 내장 property로 구현

### 인스턴스를 생성할때 반지름을 입력받는 원 객체 / 원의 속성을 가지고 있는..

> 반지름은 mutable    
> 그 외에 지름, 둘레, 넓이는 immutable



In [181]:
def no(*args):
    raise AttributeError("you can't")


class Circle:
    pi = 3.14
    
    def __init__(self, radius):
        self.radius = radius
    
    @property
    def diameter(self):
        return self.radius * 2

    @property
    def circumference(self):
        return self.radius * self.pi * 2
    
    @property
    def area(self):
        return self.radius ** 2 * self.pi

#     area = area.setter(no)
#     circumference = circumference.setter(no)

In [182]:
c = Circle(10)

In [183]:
c.area = 12

AttributeError: can't set attribute

# 내장 property 구현체

In [195]:
class my_property:
    def __init__(self, fget=None, fset=None, fdel=None):
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        
    def setter(self, fset):
        self.fset = fset
        return self 
    
    def deleter(self, fdel):
        self.fdel = fdel
        return self
    
    def getter(self, fget):
        self.fget = fget
        return self

    def __get__(self, instance, owner):
        return self.fget(instance)
    
    def __set__(self, instance, value):
        if not self.fset:
            raise AttributeError
        return self.fset(instance, value)
    
    def __delete__(self, instance):
        if not self.fdel:
            raise AttributeError
        return self.fdel(instance)

In [197]:
class OnlyInt:
    @my_property
    def data(self):
        return self._data
    
    @data.setter
    def data(self, value):
        if isinstance(value, int):
            self._data = value
        else:
            raise AttributeError("only int")
            
    @data.deleter
    def data(self):
        raise AttributeError("can't")

In [198]:
c = OnlyInt()

In [199]:
c.data = 12

In [200]:
c.data

12

In [201]:
c.data = '123'

AttributeError: only int

# 내장 디스크립터 2 : classmethod

In [206]:
class A:
    def m(arg):
        print(arg)

    @classmethod
    def cm(arg):
        print(arg)

In [207]:
a = A()

In [208]:
a.m()

In [209]:
a.cm()

<class '__main__.A'>


## 도전 : classmethod 구현

In [211]:
from functools import partial


class my_classmethod:
    def __init__(self, func):
        self.func = func
    
    def __get__(self, instance, owner):
        return partial(self.func, owner)

In [212]:
class A:
    def m(arg):
        print(arg)

    @my_classmethod
    def cm(arg):
        print(arg)

In [214]:
a = A()
a.m(), a.cm()

<class '__main__.A'>


(None, None)

# 내장 디스크립터 : staticmethod

In [215]:
class A:
    @staticmethod
    def sum(a, b, c):
        return a + b + c

In [216]:
a = A()
a.sum(1, 2, 3)

6

# 도전 : staticmethod 구현

In [217]:
class my_staticmethod:
    def __init__(self, func):
        self.func = func
        
    def __get__(self, instance, owner):
        return self.func

In [218]:
class A:
    @my_staticmethod
    def sum(a, b, c):
        return a + b + c

In [219]:
a = A()
a.sum(1, 2, 3)

6

# `__get__`, `__set__`, `__delete__`

### 3.6   하나 추가 + `__set_name__`

#### 3.6 이전

In [220]:
class A:
    def __init__(self, value, name):
        self.value = value
        self.name = name
        
    def __get__(self, instance, onwer):
        print('get', self.name)
        return self.value


class B:
    x = A(10, 'x')
    y = A(20, 'y')

In [221]:
b = B()

In [222]:
b.x

get x


10

In [223]:
b.y

get y


20

### 3.6 이후

In [228]:
class A:
    def __init__(self, value):
        self.value = value

    def __set_name__(self, instance, name):
        print("init", name)
        self.name = name
        
    def __get__(self, instance, onwer):
        print('get', self.name)
        return self.value


class B:
    x = A(10)
    y = A(20)

init x
init y


In [229]:
b = B()

In [230]:
b.x

get x


10

In [231]:
b.y

get y


20

# 디스크립터 사용예 : 장고 ORM

```
class User(django.db.Model):
    name = models.charfield(max_lengrh=12)
    location = models.textField() 
```

In [234]:
!open .